# Segmenting and Clustering Neighborhoods in Toronto

## Section 1. Build the dataframe of postal codes of Toronto Neighbourhoods

### 1.a Read table form the wikipedia page and transform it into a pandas Dataframe

In [394]:
import pandas as pd
import requests
import numpy as np

# Read table form the wikipedia page and transform it into a pandas Dataframe
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # url of the wikipedia webpage
req = requests.get(wiki_url) # request the webpage
wikitable = pd.read_html(wiki_url, attrs={"class": "wikitable"}) #read the table
codes = wikitable[0] 
codes.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True) #change the name of the column "Neighbourhood" to "Neighborhood"
codes.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 1.b Process only the cells that have an assigned Borough

In [395]:
#Only process the cells that have an assigned borough. 
#Ignore cells with a Borough that is 'Not assigned'.
codes = codes[codes.Borough !='Not assigned']
codes = codes.reset_index(drop=True)
codes.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### 1.c Combine into one row the neighborhoods with the same postal code separated with a comma 

In [396]:
# Combine into one row the neighborhoods with the same postal code, separated with a comma 
grouped_codes = codes.groupby(['Postcode','Borough']).agg({'Neighborhood': lambda x: ', '.join(x)})
grouped_codes = grouped_codes.reset_index()
grouped_codes.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 1.d For a 'Not assigned' neighborhood, the Neighbourhood will be the same as the Borough

In [397]:
# For a 'Not assigned' neighborhood, the neighboruhood will be the same as the borough
na_neig = grouped_codes.Neighborhood=='Not assigned'
grouped_codes.loc[na_neig, 'Neighborhood'] = grouped_codes.loc[na_neig, 'Borough']


### 1.e Print the number of rows of the postcodes dataframe

In [398]:
# use the .shape method to print the number of rows of your dataframe
df_codes = grouped_codes
df_codes.shape
print('Number of rows: ',df_codes.shape[0])
print('Number of columns: ',df_codes.shape[1])

Number of rows:  103
Number of columns:  3


Print the first lines of the final Data Frame

In [399]:
print(df_codes.head())

  Postcode      Borough                            Neighborhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae


## Section 2. Get the longitud and latitud of the Toronto postcodes

#### With Geocoder

_Geocoder gives a timeout error, so the provided CSV has been used instead_

#### With CSV file provided

Read the csv file

In [400]:
# Read the csv file
pc_csv = pd.read_csv('http://cocl.us/Geospatial_data')
pc_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the original DataFrame (*df_codes*) with the list of latitude and longitude from the csv file (*pc_csv*)

In [401]:
# Merge the original DataFrame (df_codes) with the list of latitude and longitude from the csv file (pc_csv)

nb_df = pd.merge(df_codes,pc_csv, left_on='Postcode', right_on='Postal Code')
nb_df = nb_df.drop(['Postal Code'],axis=1)
nb_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Section 3. Explore and cluster the neighborhoods in Toronto

Import the libraries and dependencies needed

In [402]:
# download dependencies we will need

!conda install -c conda-forge geopy --yes # install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # install folium
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Get the latitude and longitude values of Toronto using *geopy* library

In [403]:
#### Use geopy library to get the latitude and longitude values of Toronto.

address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto using *Folium* and add the neighborhoods using their longitud and latitud values

In [404]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nb_df['Latitude'], nb_df['Longitude'], nb_df['Borough'], nb_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Filter the Boroughs that containg the word *Toronto*

In [405]:
# Boroughs that contain the word Toronto

btoronto_data = nb_df[nb_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
btoronto_data['Borough'].unique() # Boroughs that contain the word Toronto

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

Map with the filtered neighborhoods

In [406]:
# Map of the neigbourhoods in the Boroughs that contain the 'Toronto' in the name
map_btoronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(btoronto_data['Latitude'], btoronto_data['Longitude'], 
                                           btoronto_data['Borough'], btoronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_btoronto)  
    
map_btoronto

### Exploring the neighborhoods with Foursquare

In [407]:
# Foursquare credentials and version

CLIENT_ID = 'XGF3ZI3JEQEIGNQG3I1TDVYUQB2FODPIAEPFLFW1RHR1CX25' # Foursquare ID
CLIENT_SECRET = 'QQHYBWYDUGYDYE3ATIFIBEKQICPHQF4RUVFL3UBZSY1XYQGK' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XGF3ZI3JEQEIGNQG3I1TDVYUQB2FODPIAEPFLFW1RHR1CX25
CLIENT_SECRET:QQHYBWYDUGYDYE3ATIFIBEKQICPHQF4RUVFL3UBZSY1XYQGK


Create a function that gets the top 'limit' venues within a defined 'radius' of each neighborhood

In [408]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=10, radius=500): 
    venues_list=[]
    
    for name,lat,lng in zip(names, latitudes,longitudes):
        #API request URL
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #get the results of the request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #construct the DataFrame venues with the relevant information
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use the getNearbyVenues function to find the 500 venues within the filtered Neighbourhoods

In [409]:
btoronto_venues=getNearbyVenues(names=btoronto_data['Neighborhood'], 
                latitudes=btoronto_data['Latitude'], 
                longitudes=btoronto_data['Longitude'], LIMIT=500)

Print the total number of venues found in the selected neighborhoods

In [410]:
print('The number of venues found is: ', btoronto_venues.shape[0]) # It gives the number of venues found

btoronto_venues.head() # first lines of the Data Frame of venues

The number of venues found is:  1719


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Group the venues found by Neighbourhood. The count number in the columns indicate the number of venues in each neighborhood.

In [411]:
btoronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


Number of the different Venue Categories

In [412]:
len(btoronto_venues['Venue Category'].unique())

237

### Explore the neighborhoods

Contruct a Data Frame matrix with the venues categories in the columns and each row representing one Neighborhood

In [413]:
# one hot encoding
btoronto_onehot = pd.get_dummies(btoronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
btoronto_onehot['Neighborhood'] = btoronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [btoronto_onehot.columns[-1]] + list(btoronto_onehot.columns[:-1])
btoronto_onehot = btoronto_onehot[fixed_columns]

btoronto_onehot.head()

btoronto_grouped = btoronto_onehot.groupby('Neighborhood').mean().round(3).reset_index() 
btoronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.03,...,0.0,0.0,0.0,0.0,0.020,0.0,0.0,0.01,0.0,0.0
1,Berczy Park,0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,...,0.0,0.0,0.0,0.0,0.018,0.0,0.0,0.00,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.000,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.053,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.00,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000,0.0,0.067,0.067,0.067,0.067,0.2,0.133,0.00,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.0


Top 5 venues in each Neighborhood

In [414]:
# Each neighborhood with the top 5 venues
num_top_venues = 5

# For each Neighborhood print the 5 top venues and their frequencies
for hood in btoronto_grouped['Neighborhood']:
    print("----"+hood+"----") # prints the name of the Neighborhood
    
    temp = btoronto_grouped[btoronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.05
2          Bar  0.04
3   Steakhouse  0.04
4   Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
                   venue  freq
0         Breakfast Spot  0.09
1                   Café  0.09
2            Coffee Shop  0.09
3      Convenience Store  0.04
4  Performing Arts Venue  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.10
1         Yoga Studio  0.05
2          Restaurant  0.05
3          Smoke Shop  0.05
4         Pizza Place  0.05


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.20
1  Airport Te

Create a dataframe with the most common venues per each Neighborhood. The most common venues are ordered per columns ('1st Most Common Venue', '2nd Most Common Venue',..etc).

In [415]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = btoronto_grouped['Neighborhood']

for ind in np.arange(btoronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(btoronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,American Restaurant,Burger Joint,Cosmetics Shop,Restaurant,Thai Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Seafood Restaurant,Cheese Shop,Café,Farmers Market,Jazz Club
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Intersection,Performing Arts Venue,Pet Store,Grocery Store,Climbing Gym,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Boat or Ferry,Airport Lounge,Coffee Shop,Harbor / Marina,Sculpture Garden,Airport Gate,Airport Food Court,Airport


## Cluster Neighbourhoods 

Run *k*-means to cluster the neighborhood into 5 clusters.

In [416]:
# set number of clusters
kclusters = 5

btoronto_grouped_clustering = btoronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(btoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [417]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

btoronto_merged = btoronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
btoronto_merged = btoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

btoronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Other Great Outdoors,Health Food Store,Trail,Pub,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Wings Joint
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Sandwich Place,Pet Store,Movie Theater,Italian Restaurant,Fish & Chips Shop,Intersection,Sushi Restaurant,Brewery,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Sandwich Place,Bar,Stationery Store,Fish Market,Coworking Space
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Swim School,Bus Line,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Localize each cluster in the map

In [418]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(btoronto_merged['Latitude'], btoronto_merged['Longitude'], btoronto_merged['Neighborhood'], btoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

Cluster 1

In [419]:
btoronto_merged.loc[btoronto_merged['Cluster Labels'] == 0, btoronto_merged.columns[[1] + list(range(5, btoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Other Great Outdoors,Health Food Store,Trail,Pub,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Wings Joint
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant
2,East Toronto,0,Park,Sandwich Place,Pet Store,Movie Theater,Italian Restaurant,Fish & Chips Shop,Intersection,Sushi Restaurant,Brewery,Food & Drink Shop
3,East Toronto,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Sandwich Place,Bar,Stationery Store,Fish Market,Coworking Space
5,Central Toronto,0,Gym,Clothing Store,Restaurant,Food & Drink Shop,Dance Studio,Hotel,Sandwich Place,Breakfast Spot,Park,Eastern European Restaurant
6,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Health & Beauty Service,Italian Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Park,Mexican Restaurant
7,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Restaurant,Gym,Italian Restaurant,Coffee Shop,Café,Thai Restaurant
9,Central Toronto,0,Coffee Shop,Pub,Health & Beauty Service,American Restaurant,Restaurant,Sushi Restaurant,Liquor Store,Fried Chicken Joint,Sports Bar,Bagel Shop
11,Downtown Toronto,0,Coffee Shop,Bakery,Restaurant,Café,Pub,Market,Pizza Place,Italian Restaurant,Bank,Playground
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Pizza Place,Men's Store,Mediterranean Restaurant,Hotel


Cluster 2

In [420]:
btoronto_merged.loc[btoronto_merged['Cluster Labels'] == 1, btoronto_merged.columns[[1] + list(range(5, btoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Playground,Tennis Court,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 3

In [421]:
btoronto_merged.loc[btoronto_merged['Cluster Labels'] == 2, btoronto_merged.columns[[1] + list(range(5, btoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Pool,Garden,Wings Joint,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 4

In [422]:
btoronto_merged.loc[btoronto_merged['Cluster Labels'] == 3, btoronto_merged.columns[[1] + list(range(5, btoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Park,Swim School,Bus Line,Wings Joint,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Cluster 5

In [423]:
btoronto_merged.loc[btoronto_merged['Cluster Labels'] == 4, btoronto_merged.columns[[1] + list(range(5, btoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Park,Playground,Trail,Building,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
23,Central Toronto,4,Sushi Restaurant,Park,Trail,Jewelry Store,Wings Joint,Donut Shop,Discount Store,Dog Run,Doner Restaurant,Eastern European Restaurant


This clustering analysis based on the venues results in one big cluster (cluster number 1) and four small clusters with only one or two elements.